In [1]:
from pynq import Overlay
from pynq import MMIO
from pynq.lib.iic import AxiIIC
import cffi, time
import csv
import serial.tools.list_ports
import numpy as np

overlay = Overlay("/home/xilinx/pynq/overlays/axi_test.bit")

In [2]:
# i2c MMIO 설정
ADDR_BASE = 0x41600000
ADDR_RANGE = 0x10000
i2c_obj = MMIO(ADDR_BASE,ADDR_RANGE)

In [3]:
i2c_obj.write(292,0)

iic_ip = overlay.ip_dict['axi_iic_0']
iic = AxiIIC(iic_ip)

ffi = cffi.FFI()

In [4]:
def read_(addr,reg,length=1):
    iic.send(addr,bytes([reg]),1,0)
    buf = ffi.new("unsigned char[]",length)
    iic.receive(addr, buf, length, 0)
    return [buf[i] for i in range(length)]

def write_(addr, reg, value):
    iic.send(addr, bytes([reg,value]), 2, 0)

def to_int16(low,high):
    val = (high << 8) | low
    if val & 0x8000:
        val -= 1 << 16
    return val

In [5]:
# BN0055 connection check
BN0055_addr = 0x28
chip_addr = 0x00
chip_id = read_(BN0055_addr, 0x00, 1)[0]
print("CHIP ID:", hex(chip_id)) #0xA0

#NDOF mode on 0x0C
write_(BN0055_addr, 0x3D, 0X0C)
time.sleep(0.1)

#calibration check
cal = read_(BN0055_addr, 0x35, 1)[0]
sys = (cal >> 6) & 0x03
gyro = (cal >> 4) & 0x03
accel = (cal >> 2) & 0x03
mag = cal & 0x03

print(f"Cal : sys = {sys} Gyro = {gyro} accel = {accel} mag = {mag}")

CHIP ID: 0xa0
Cal : sys = 0 Gyro = 3 accel = 0 mag = 3


In [6]:
# roboclaw encoder open

ports = serial.tools.list_ports.comports()

for port in ports:
    print(f"Device: {port.device} | Description: {port.description}")

import sys
sys.path.append("/home/xilinx/pynq/roboclaw")  # 실제 roboclaw_3.py가 있는 경로
from roboclaw_3 import Roboclaw

rc = Roboclaw("/dev/ttyACM0", 38400)  # 또는 ttyUSB0
rc.Open()

rc_address = 0x80
rc.ForwardM1(rc_address, 20)
rc.ForwardM2(rc_address, 20)

Device: /dev/ttyACM0 | Description: USB Roboclaw 2x30A


True

In [7]:
# ====== PID Controller ======
class PIDController:
    def __init__(self, Kp, Ki, Kd, output_limits):
        self.Kp, self.Ki, self.Kd = Kp, Ki, Kd
        self.output_limits = output_limits
        self.integral_sum = 0
        self.previous_error = 0

    def update(self, error, dt):
        self.integral_sum += error * dt
        # Anti-windup
        self.integral_sum = np.clip(self.integral_sum, self.output_limits[0], self.output_limits[1])

        derivative = (error - self.previous_error) / dt if dt > 0 else 0
        output = self.Kp * error + self.Ki * self.integral_sum + self.Kd * derivative
        self.previous_error = error
        return np.clip(output, self.output_limits[0], self.output_limits[1])


In [8]:
# ====== Robot Constants ======
R = 0.065   # wheel radius (m)
L = 0.35    # wheel distance (m)
pi = 3.142
CPR = 1288

# ====== State Variables ======
prev_x, prev_y = 0.0, 0.0
target_x, target_y = 15.0, 0.0
base_linear_speed = 0.4  # m/s
prev_encL, prev_encR = rc.ReadEncM1(rc_address)[1], rc.ReadEncM2(rc_address)[1]

In [9]:
# ====== PID Controllers ======
speed_pid_left = PIDController(0.5, 0.1, 0.05, (0, 127))
speed_pid_right = PIDController(0.5, 0.1, 0.05, (0, 127))
heading_pid = PIDController(1.0, 0.01, 0.1, (-0.5, 0.5))

In [10]:
# ====== Timing ======
inner_dt = 0.01   # 100Hz inner loop
outer_dt = 0.05   # 20Hz outer loop
last_outer_time = time.time()

In [11]:
# ====== CSV Logging Setup ======
log_filename = "robot_log.csv"
with open(log_filename, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([
        "time", "cmd_l", "cmd_r", "vl", "vr", "v_robot", "w_robot",
        "curr_x", "curr_y", "desired_heading(rad)", "current_heading(rad)",
        "heading_error(rad)", "left_target_speed", "right_target_speed"
    ])

In [12]:
# ====== Helper Functions ======
def cal_v(enc_l, enc_r, dt):
    global prev_encL, prev_encR

    delta_encL = -(enc_l - prev_encL)
    delta_encR = -(enc_r - prev_encR)
    prev_encL, prev_encR = enc_l, enc_r

    vl = 2 * pi * R * (delta_encL / (dt * CPR))
    vr = 2 * pi * R * (delta_encR / (dt * CPR))
    return vl, vr


def current_locate(v_robot, w_robot, dt, heading):
    """Update current position based on previous state and velocity"""
    global prev_x, prev_y
    curr_x = prev_x + v_robot * np.cos(heading) * dt
    curr_y = prev_y + v_robot * np.sin(heading) * dt
    prev_x, prev_y = curr_x, curr_y
    return curr_x, curr_y

In [ ]:
# ====== Main Control Loop ======
prev_time = time.time()



while True:
    start_time = time.time()
    dt_loop = start_time - prev_time
    prev_time = start_time

    # --- 1. Read Sensors ---
    enc_l, enc_r = rc.ReadEncM1(rc_address)[1], rc.ReadEncM2(rc_address)[1]
    data = read_(BN0055_addr, 0x1A, 2)  # Heading 데이터만 읽기
    current_heading = to_int16(data[0], data[1]) / 16.0
    current_heading = np.deg2rad(current_heading)
  

    # --- 2. Compute velocities ---
    vl, vr = cal_v(enc_l, enc_r, inner_dt)
    v_robot = (vl + vr) / 2
    w_robot = (vr - vl) / L

    # --- 3. Update odometry ---
    curr_x, curr_y = current_locate(v_robot, w_robot, inner_dt, current_heading)

    # --- 4. OUTER LOOP (20Hz) ---
    if time.time() - last_outer_time >= outer_dt:
        desired_heading = np.arctan2(target_y - curr_y, target_x - curr_x)
        heading_error = desired_heading - current_heading
        heading_error = np.arctan2(np.sin(heading_error), np.cos(heading_error))
        

        # Heading control (angular velocity)
        steering_correction_omega = heading_pid.update(heading_error, outer_dt)

        # Compute target wheel speeds
        left_target_speed = (base_linear_speed - steering_correction_omega * L / 2)
        right_target_speed = (base_linear_speed + steering_correction_omega * L / 2)

        last_outer_time = time.time()  # update last outer loop time

    # --- 5. INNER LOOP (100Hz) ---
    cmd_l = speed_pid_left.update(left_target_speed - vl, inner_dt)
    cmd_r = speed_pid_right.update(right_target_speed - vr, inner_dt)
    cmd_l = int(cmd_l)
    cmd_r = int(cmd_r)

    # --- 6. Apply PWM to motors ---
    rc.ForwardM1(rc_address, cmd_l)
    rc.ForwardM2(rc_address, cmd_r)

        # === Save log each loop ===
    with open(log_filename, mode="a", newline="") as file:
        writer = csv.writer(file)
        writer.writerow([
            time.time(), cmd_l, cmd_r, vl, vr, v_robot, w_robot,
            curr_x, curr_y, desired_heading, current_heading,
            heading_error, left_target_speed, right_target_speed
        ])

    if np.hypot(target_x - curr_x, target_y - curr_y) < 0.05:
        rc.ForwardM1(rc_address, 0)
        rc.ForwardM2(rc_address, 0)
        break

    # --- 7. Maintain 100Hz Loop ---
    elapsed = time.time() - start_time
    time.sleep(max(0, inner_dt - elapsed))